In [1]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
from entsoe import EntsoePandasClient
import pandas as pd
import json
import requests
from pandas import json_normalize
from bs4 import BeautifulSoup
from shapely.geometry import Polygon, LineString
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from datetime import datetime
from scipy import stats
from pathlib import Path
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path

# geo libraries
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt
from geopy.distance import geodesic
from geopandas import GeoDataFrame
import shapely.ops
import shapely.geometry
from shapely.geometry import Polygon, LineString
cwd = Path().resolve()

import sqlite3
# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt
s = requests.Session()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#------
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
##--

class InvalidCoordinate(Exception):
    pass

In [2]:
conn = sqlite3.connect('file.db')
vessel = pd.read_sql_query("SELECT * FROM lng", conn) #= pd.read_csv('./data/lng_tankers_lat.csv', index_col=False)
vessel.dropna(subset=['imo'], inplace=True)
vessel.head()

,imo,ship,build,GT,DWT,size,type,lat,lon,date
0,9388819,LIJMILIYA,2009,168189,155159,345 / 55,LNG Tanker,6.69627,77.42957,2022-12-20
1,9372743,AL GHUWAIRIYA,2008,168189,154940,345 / 55,LNG Tanker,6.42555,95.11727,2022-12-20
2,9388821,AL SAMRIYA,2009,168189,154900,345 / 55,LNG Tanker,39.92342,-10.14878,2022-12-20
3,9388833,BU SAMRA,2008,163922,130442,345 / 54,LNG Tanker,6.25460,95.26184,2022-12-20
4,9397315,AL MAFYAR,2009,163922,130441,345 / 54,LNG Tanker,1.39792,104.46966,2022-12-20


In [3]:
"""
SQLITE: DELETE FROM lng WHERE date = '2022-12-28'

interesting news shipyard: https://www.ship-technology.com/news/daewoo-shipbuilding-contract-lng-ships/

"""
def get_all_lng_carriers_from_vesselfinder():
    """ """
    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    df2 = pd.DataFrame()
    for i in range(1,40):
        try:
            url = f'https://www.vesselfinder.com/de/vessels?page={i}&type=604'
            r = requests.get(url, headers=header)
            r.raise_for_status()
            df = pd.read_html(r.text, extract_links='all')[0]
            df.rename(columns={df.columns[0]: 'imo'}, inplace=True)
            df['imo'] = df.apply(lambda x: x['imo'][1], axis=1)
            df['imo']= df['imo'].str.split("/").str[-1]
            
            
            df.rename(columns={df.columns[1]: 'ship'},inplace=True)
            df['ship'] = df.apply(lambda x: x['ship'][0], axis=1)
    
            df.rename(columns={df.columns[2]: 'build'},inplace=True)
            df['build'] = df.apply(lambda x: x['build'][0], axis=1)
            
            df.rename(columns={df.columns[3]: 'GT'},inplace=True)
            df['GT'] = df.apply(lambda x: x['GT'][0], axis=1)

            df.rename(columns={df.columns[4]: 'DWT'},inplace=True)
            df['DWT'] = df.apply(lambda x: x['DWT'][0], axis=1)
            
            df.rename(columns={df.columns[5]: 'size'},inplace=True)
            df['size'] = df.apply(lambda x: x['size'][0], axis=1)
            # df["size"]= df["size"].str.split(" / ")
            
            df[['ship', 'type']] = df["ship"].apply(lambda x: pd.Series(str(x).split("  LNG Tanker")))
            df['type'] = "LNG Tanker"
            
            df2 = df2.append(df, ignore_index=True)
            
        except Exception as e:
            print(e)
            break
    
    # temporary save to csv file
    df2.to_csv('./data/lng_tankers.csv', index=False)
    return df2


def get_current_vessel_location(imo):
    try:
        url = f"https://www.vesselfinder.com/en/vessels/VOS-TRAVELLER-IMO-{imo}"
        r = s.get(url, headers=header)
        r.raise_for_status()
        soup = BeautifulSoup(r.text)
        data = json.loads(soup.find("div", {"id": "djson"})['data-json'])
        time.sleep(0.1)
        lat = data['ship_lat']
        lon = data['ship_lon']
        
        # The latitude must be a number between -90 and 90 and the longitude between -180 and 180.
        if (lat < -90 or lat > 90) or (lon < -180 or lon > 180):
            raise InvalidCoordinate
        
        return pd.Series([lat, lon, datetime.now().strftime("%Y-%m-%d")])
    
    except InvalidCoordinate:
        print(f"Invalid IMO: {imo}")
        return pd.Series([np.nan, np.nan, datetime.now().strftime("%Y-%m-%d")])
    
    except requests.exceptions.Timeout:
        print("dfdf")
        # Maybe set up for a retry, or continue in a retry loop
    except requests.exceptions.TooManyRedirects:
        print("dddddddddd")
        # Tell the user their URL was bad and try a different one
    except requests.exceptions.RequestException as e:
        print("dfdsafdfadfadfadf")
    
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        input("change vpn V2")
        return pd.Series([np.nan, np.nan, datetime.now().strftime("%Y-%m-%d")])

    except Exception as e:
        input("change vpn")
        return pd.Series([np.nan, np.nan, datetime.now().strftime("%Y-%m-%d")])


#def run():
print(f"start length: {vessel.shape[0]}")
t0 = time.time()
conn = sqlite3.connect('file.db')
df = pd.read_sql_query("SELECT * FROM lng", conn)
df.drop_duplicates(subset="imo", keep='first', inplace=True)
df.dropna(subset=['imo'], inplace=True)
df[['lat','lon','date']] = df.apply(lambda x: get_current_vessel_location(x['imo']), axis=1)
print(f"Processing took: {round(time.time()-t0)} s") # ~6.4 min
vessel = vessel.append(df)
print(f"final length after appending: {vessel.shape[0]}")
vessel.to_sql('lng', conn, if_exists='replace', index = False)
conn.close()
df.tail()


start length: 2076
Invalid IMO: 9338266
Invalid IMO: 9918004
Invalid IMO: 9778923
Invalid IMO: 9761815
Invalid IMO: 9903437
Invalid IMO: 9903449
Invalid IMO: 9903425
Invalid IMO: 7400704
Invalid IMO: 9246578
Invalid IMO: 9165011
Invalid IMO: 7391197
Invalid IMO: 7391202
Invalid IMO: 7359955
Invalid IMO: 7382732
Invalid IMO: 9030840
Invalid IMO: 7357452
Invalid IMO: 7413232
Invalid IMO: 7390167
Invalid IMO: 7390179
Invalid IMO: 7390181
Invalid IMO: 7619575
Invalid IMO: 7619587
Invalid IMO: 7390155
Invalid IMO: 7390208
Invalid IMO: 7390193
Invalid IMO: 7428469
Invalid IMO: 7428445
Invalid IMO: 7428471
Invalid IMO: 7702401
Invalid IMO: 7708948
Invalid IMO: 7400663
Invalid IMO: 8013950
Invalid IMO: 8014473
Invalid IMO: 9020766
Invalid IMO: 7360136
Invalid IMO: 7360124
Invalid IMO: 8125832
Invalid IMO: 7359670
Invalid IMO: 8014409
Invalid IMO: 7361934
Invalid IMO: 7400675
Invalid IMO: 7391214
Invalid IMO: 7235939
Invalid IMO: 7217896
Invalid IMO: 7121633
Invalid IMO: 7347794
Invalid IMO: 73

,imo,ship,build,GT,DWT,size,type,lat,lon,date
687,9469235,KAKUREI MARU,2008,2952,1801,86 / 15,LNG Tanker,40.60177,141.57677,2022-12-30
688,9433884,SHINJU MARU NO.2,2008,2930,1781,86 / 15,LNG Tanker,33.97234,131.01181,2022-12-30
689,9260603,SHINJU MARU NO.1,2003,2936,1781,86 / 15,LNG Tanker,33.92027,130.85628,2022-12-30
690,9762039,HANTONG CNG 001,2022,6205,1280,110 / 18,LNG Tanker,32.12058,119.89967,2022-12-30
691,9275074,PIONEER KNUTSEN,2004,1687,817,69 / 12,LNG Tanker,65.43509,11.93226,2022-12-30


In [ ]:
### this is the part not yet working ;)

In [ ]:
#vessel = pd.read_csv('./data/lng_tankers_lat.csv', index_col=False)
vessel = pd.read_sql_query("SELECT * FROM lng", conn)
vessel.sort_values(by='DWT', ascending=False, inplace=True)

geometry = [Point(xy) for xy in zip(vessel.lon, vessel.lat)]
# vessel = vessel.drop(['lon', 'lat'], axis=1)
# vessel.to_sql('lng', conn, if_exists='replace', index = False)
vessel = GeoDataFrame(vessel, crs="EPSG:4326", geometry=geometry)
# not needed any more: vessel['date'] =  datetime.strptime('2022-12-20', '%Y-%m-%d') # datetime.now().strftime("%Y-%m-%d")
vessel.head()

In [ ]:
df_m3 = pd.read_html('http://www.aukevisser.nl/supertankers/gas-SP/id703.htm', skiprows = 0, header=0)[2]
# list of fleet: https://www.nakilat.com/fleet-list/
df_m3.to_csv('./data/lng_m3_without_imo.csv', index=False)
# df_m3[df_m3['Ship Name'].str.lower() =='LIJMILIYA'.lower()].head()
df_m3.head()

In [ ]:
def test(name):
    # name_list = name.split(' ')
    # name_list = [i if len(i) >3 else None for i in name_list]
    # print(name_list)
    name = name.lower().replace('lng', '').replace(' al', '').replace('al ', '').replace('gas ', '').replace(' gas', '')
    for index, row in df_m3.iterrows():    
        #ship_name = row['Ship Name'].split(' ')
        #print(SequenceMatcher(a=name,b=row['Ship Name']).ratio())
        try:
            if SequenceMatcher(a=name,b=row['Ship Name'].lower()).ratio() > 0.4:
                print(name, row["Ship Name"].lower(), SequenceMatcher(a=name,b=row['Ship Name'].lower()).ratio())
                # return row["Ship Name"]
            #    return pd.Series([row["Ship Name"], row["Capacity (cu.m.)"]])
                return pd.Series([np.nan, np.nan])
            else:
                pass
        except AttributeError:
            # print(row['Ship Name'])
        except (TypeError, ValueError):
            pass
    return pd.Series([np.nan, np.nan])
        
vessel[['found','Capacity']] = vessel.apply(lambda x: test(x['ship']), axis=1)
vessel.head(100)